### 22i-0781 - Ahmad Ryan - Lab 8

## Task 1

In [ ]:
class SudokuSolver:
    def __init__(self, grid):
        self.grid = grid
        self.variables = [(i, j) for i in range(9) for j in range(9) if grid[i][j] == 0]
        self.domains = {
            var: set(range(1, 10)) if grid[var[0]][var[1]] == 0 else {grid[var[0]][var[1]]}
            for var in self.variables
        }
        self.constraints = self.generate_constraints()

    def generate_constraints(self):
        """
        Generate row, column, and 3x3 subgrid constraints.
        """
        constraints = set()
        
        for i in range(9):
            for j in range(9):
                if self.grid[i][j] == 0:
                    for k in range(9):
                        if k != j:
                            constraints.add(((i, j), (i, k)))  # Row constraint
                        if k != i:
                            constraints.add(((i, j), (k, j)))  # Column constraint
                    
                    start_row, start_col = (i // 3) * 3, (j // 3) * 3
                    for r in range(start_row, start_row + 3):
                        for c in range(start_col, start_col + 3):
                            if (r, c) != (i, j):
                                constraints.add(((i, j), (r, c)))  # Subgrid constraint
        
        return constraints

    def is_valid(self, var, value):
        """
        Check if assigning 'value' to 'var' satisfies all constraints.
        """
        row, col = var
        for (v1, v2) in self.constraints:
            if v1 == var:
                r, c = v2
                if self.grid[r][c] == value:
                    return False
        return True

    def forward_checking(self, var, value):
        """
        Remove 'value' from the domain of neighbors of 'var'.
        Return list of affected variables to restore later.
        """
        affected = []
        for (v1, v2) in self.constraints:
            if v1 == var and v2 in self.variables and value in self.domains[v2]:
                self.domains[v2].remove(value)
                affected.append(v2)
        return affected

    def restore_domains(self, affected, value):
        """
        Restore removed values after backtracking.
        """
        for var in affected:
            self.domains[var].add(value)

    def select_unassigned_variable(self):
        """
        Select the variable with the minimum remaining values (MRV heuristic).
        """
        return min(self.variables, key=lambda var: len(self.domains[var]))

    def backtrack(self):
        """
        Solve the puzzle using backtracking + MRV + forward checking.
        """
        if not self.variables:
            return True  # Solved
        
        var = self.select_unassigned_variable()
        domain_values = list(self.domains[var])
        
        for value in domain_values:
            if self.is_valid(var, value):
                self.grid[var[0]][var[1]] = value
                self.variables.remove(var)
                
                affected = self.forward_checking(var, value)
                if self.backtrack():
                    return True
                
                self.grid[var[0]][var[1]] = 0
                self.variables.append(var)
                self.restore_domains(affected, value)
        
        return False

    def solve(self):
        if self.backtrack():
            return self.grid
        else:
            return None


In [4]:

# 9x9 Sudoku Grid (0 represents empty cells)
sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

# Solve the Sudoku
solver = SudokuSolver(sudoku_grid)
solution = solver.solve()

# Print the solution
if solution:
    print("Solved Sudoku:")
    for row in solution:
        print(row)
else:
    print("No solution exists.")


Solved Sudoku:
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
